## Get MLB batting stats from 2007 to 2021

In [3]:
import pandas as pd
import csv
from pybaseball import batting_stats
import import_ipynb
from data_cleaning import *

importing Jupyter notebook from data_cleaning.ipynb
(1669, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1669 non-null   int64  
 1   name             1669 non-null   object 
 2   position         1669 non-null   object 
 3   age              1669 non-null   float64
 4   from_team        1669 non-null   object 
 5   to_tam           1668 non-null   object 
 6   contract_length  1669 non-null   object 
 7   total_salary     1275 non-null   float64
 8   avg_salary       1275 non-null   float64
 9   year             1669 non-null   int64  
 10  spotracID        1669 non-null   int64  
 11  spotracLink      1669 non-null   object 
dtypes: float64(3), int64(3), object(6)
memory usage: 156.6+ KB
Total rows left: 1275
Players with the same name:
# of players to be searched: 383
Gathering player lookup table. This may

In [4]:
# import mlb all batting stats from 2007-2021
batting_df = pd.read_csv('mlb-batting.csv')
pd.set_option('display.max_columns', None)
print(batting_df.shape)
batting_df.head()

# drop rows with AB < 400
# batting_df = batting_df.drop(batting_df[batting_df['AB']<400].index)
# batting_df.shape

(14118, 320)


,Unnamed: 0,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,292,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,47,5,32,129,80,81,8,91,8,5,0,5,30,6,0.346,147,195,92,18,2582,1095,1487,14,0,0,0.132,0.148,0.89,0.438,0.640,1.078,0.294,0.368,0.0075,0.212,0.339,0.449,0.092,0.164,0.095,0.0,0.449,67.2,139,62.4,16.8,18.4,-5.3,101.1,10.4,$83.3,7.0,185,5.77,-8.16,13.93,62.93,6.11,0.92,3.62,4,6.44,-0.15,0.547,93.2,0.199,84.5,0.050,88.7,0.073,79.1,0.114,84.4,0.017,85.4,NaN,NaN,0.009,NaN,40.1,11.2,4.2,1.1,11.1,2.3,NaN,2.84,2.19,3.28,0.58,3.79,5.06,NaN,0.198,0.570,0.356,0.706,0.930,0.859,0.426,0.562,0.050,6.9,0.350,0.097,0.047,0.022,NaN,0.101,0.204,0.058,0.015,NaN,0.106,NaN,NaN,NaN,93.5,92.8,88.4,85.2,NaN,92.1,84.6,78.4,80.8,NaN,84.4,NaN,NaN,-2.4,-6.1,0.4,-4.1,NaN,-1.6,1.6,2.9,4.5,NaN,0.3,NaN,NaN,9.6,6.0,4.9,3.2,NaN,5.4,1.2,-5.3,-5.8,NaN,4.5,NaN,NaN,18.3,13.3,3.1,2.5,NaN,7.1,9.8,1.5,2.5,NaN,10.1,NaN,NaN,2.03,5.33,2.58,4.34,NaN,2.72,1.87,1.03,6.59,NaN,3.72,NaN,NaN,0.158,0.546,0.355,0.635,0.919,0.857,0.509,NaN,11.6,3.1,2.7,25 - 25,69.3,1.9,1.1,0.472,0.348,0.180,0.122,0.433,0.445,0.332,0.105,NaN,0.070,0.366,0.046,0.026,NaN,NaN,0.173,0.201,0.0,84.6,NaN,79.0,93.6,88.6,84.6,NaN,NaN,92.6,84.7,93.5,0.4,NaN,3.6,-1.9,0.5,-3.2,NaN,NaN,-4.2,2.1,-6.4,3.1,NaN,-7.2,8.2,4.2,1.6,NaN,NaN,4.4,-0.2,5.9,10.5,NaN,0.9,18.2,4.0,4.0,NaN,NaN,21.4,10.5,0.0,3.82,NaN,0.48,1.91,3.30,5.95,NaN,NaN,4.75,2.00,2.04,0.162,0.539,0.355,0.647,0.917,0.857,0.512,NaN,NaN,139.0,159,68,137.0,154.0,176.0,125.0,1.00,80.0,123.0,129.0,115.0,102.0,73.0,69.0,92.0,125.0,92.3,18.5,57.0,0.131,110.6,217.0,0.5,434,0.220,0.270,NaN,NaN,NaN
1,489,10155,2013,Mike Trout,LAA,21,157,589,716,190

In [5]:
# drop all unwanted stats

var_list_basic = ['IDfg', 
            'Season', 
            'Name', 
            'Team', 
            'Age', 
            'G', 
            'PA', 
            'AB', 
            'R', 
            'H',
            '2B',
            '3B',
            'HR',
            'RBI',
            'SB',
            'CS',
            'BB',
            'SO',
            'GDP',
            'HBP',
            'SH',
            'SF',
            'IBB',
            'AVG',
            'OBP',
            'SLG',
            'BABIP',
            'ISO',
            'OPS',
            'wRC',
            'wRAA',
            'wOBA',
            'wRC+',
            'WPA',
            'WAR']
# not the full list
batting_basic_df = batting_df[var_list_basic]
batting_basic_df = batting_basic_df.sort_values(by=['IDfg', 'Season'], ascending=True)
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df.columns

Index(['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'PA', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'GDP', 'HBP', 'SH',
       'SF', 'IBB', 'AVG', 'OBP', 'SLG', 'BABIP', 'ISO', 'OPS', 'wRC', 'wRAA',
       'wOBA', 'wRC+', 'WPA', 'WAR'],
      dtype='object')

In [6]:
# re_index batting_basic_df
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df = batting_basic_df.set_index(['IDfg'])
batting_basic_df.sort_index(level=['IDfg'], inplace=True)

In [19]:
# aggregate batting_basic based on FA year
agg_method = {'Age':'max',
             'G':'sum',
             'PA':'sum',
             'AB':'sum',
             'R':'sum',
             'H':'sum',
             '2B':'sum',
             '3B':'sum',
             'HR':'sum',
             'RBI':'sum',
             'SB':'sum',
             'CS':'sum',
             'BB':'sum',
             'SO':'sum',
             'GDP':'sum',
             'HBP':'sum',
             'SH':'sum',
             'SF':'sum',
             'IBB':'sum',
             'AVG':'mean',
             'OBP':'mean',
             'SLG':'mean',
             'BABIP':'mean',
             'ISO':'mean',
             'OPS':'mean',
             'wRC':'sum',
             'wRAA':'sum',
             'wRC+':'sum',
             'WPA':'sum',
             'WAR':'sum'}
batting_aggDF = pd.DataFrame()
for row in salaryFinalDF.itertuples():
    selected_years = [row.year_fa-1, row.year_fa-2, row.year_fa-3, row.year_fa-4, row.year_fa-5,]
    player = batting_basic_df.loc[row.IDfg]
    player_add = player[player['Season'].isin(selected_years)].groupby(by=['IDfg','Name']).agg(agg_method)
    player_add['Year_FA']=row.year_fa
    player_add['Salary']=row.avg_salary
    player_add['num_seaIncluded']=len(player[player['Season'].isin(selected_years)].index)
    batting_aggDF = batting_aggDF.append(player_add)

In [22]:
batting_aggDF['num_seaIncluded'].value_counts()

5    507
4     44
3      6
2      4
Name: num_seaIncluded, dtype: int64

In [24]:
batting_aggDF

Age    G    PA    AB    R    H   2B  3B   HR  RBI  \
IDfg  Name                                                                     
5677  A.J. Ellis            35  468  1713  1448  143  341   63   2   35  172   
746   A.J. Pierzynski       36  664  2587  2424  254  682  122   7   74  300   
      A.J. Pierzynski       35  664  2628  2455  272  695  129   7   70  290   
      A.J. Pierzynski       37  628  2414  2258  222  616  112   7   66  288   
      A.J. Pierzynski       38  613  2347  2191  217  610  107   8   66  281   
...                        ...  ...   ...   ...  ...  ...  ...  ..  ...  ...   
11368 Yasmani Grandal       29  638  2326  2000  254  471   95   4  104  294   
      Yasmani Grandal       30  663  2515  2136  286  512  102   5  117  322   
2530  Yonder Alonso         30  582  2118  1888  235  500  104   2   53  226   
8585  Yuniesky Betancourt   30  647  2498  2356  238  604  126  15   49  282   
2616  Zack Cozart           31  594  2390  2169  297  553  110  18   65  242   

                           SB  CS   BB   SO  GDP  HBP  SH  SF  IBB     AVG  \
IDfg  Name                                                                   
5677  A.J. Ellis            2   3  214  311   52   17  19  15   20  0.2306   
746   A.J. Pierzynski       5   6  101  278   76   29  12  21   21  0.2814   
      A.J. Pierzynski       5   5  109  273   76   28  15  21   24  0.2832   
      A.J. Pierzynski       4   6   91  280   71   33  10  22   17  0.2716   
      A.J. Pierzynski       1   4   95  278   73   34   4  23   17  0.2776   
...                        ..  ..  ...  ...  ...  ...  ..  ..  ...     ...   
11368 Yasmani Grandal       6   6  299  577   56    9   2  16    4  0.2350   
      Yasmani Grandal       8   7  350  601   65   12   2  15    5  0.2392   
2530  Yonder Alonso        19   7  204  323   54    9   0  17   16  0.2644   
8585  Yuniesky Betancourt  13  15   86  238   79    5  22  29    4  0.2526   
2616  Zack Cozart          17   4  164  372   49   16  20  21    9  0.2564   

                              OBP     SLG   BABIP     ISO     OPS  wRC  wRAA  \
IDfg  Name                                                                     
5677  A.J. Ellis           0.3340  0.3466  0.2680  0.1164  0.6804  183  -8.4   
746   A.J. Pierzynski      0.3154  0.4288  0.2898  0.1476  0.7440  301   6.0   
      A.J. Pierzynski      0.3184  0.4270  0.2922  0.1440  0.7452  305   0.0   
      A.J. Pierzynski      0.3068  0.4112  0.2840  0.1398  0.7180  268  -2.4   
      A.J. Pierzynski      0.3146  0.4196  0.2904  0.1422  0.7342  274  12.1   
...                           ...     ...     ...     ...     ...  ...   ...   
11368 Yasmani Grandal      0.3352  0.4412  0.2742  0.2060  0.7764  309  41.5   
      Yasmani Grandal      0.3458  0.4546  0.2746  0.2154  0.8004  359  58.9   
2530  Yonder Alonso        0.3336  0.4028  0.2912  0.1384  0.7366  255  12.3   
8585  Yuniesky Betancourt  0.2778  0.3858  0.2594  0.1332  0.6632  224 -67.4   
2616  Zack Cozart          0.3110  0.4226  0.2768  0.1660  0.7334  267  -4.9   

                           wRC+   WPA   WAR  Year_FA      Salary  \
IDfg  Name                                                         
5677  A.J. Ellis            464 -1.46   2.6     2017   2500000.0   
746   A.J. Pierzynski       473 -2.58   8.8     2014    500000.0   
      A.J. Pierzynski       468 -2.85   9.7     2013   7500000.0   
      A.J. Pierzynski       452 -4.54   5.3     2015   2000000.0   
      A.J. Pierzynski       483 -2.46   5.1     2016   3000000.0   
...                         ...   ...   ...      ...         ...   
11368 Yasmani Grandal       574 -1.00  22.3     2019  18250000.0   
      Yasmani Grandal       585  1.48  25.0     2020  18250000.0   
2530  Yonder Alonso         523  3.64   4.4     2018   8000000.0   
8585  Yuniesky Betancourt   374 -7.71  -2.8     2013    900000.0   
2616  Zack Cozart           469 -3.56  11.0     2018  12666667.0   

                           num_seaIncluded  
